# Geographic Data with Basemap

Un tipo común de visualización en ciencia de datos es el de datos geográficos.
La herramienta principal de Matplotlib para este tipo de visualización es el kit de herramientas Basemap, que es uno de los varios kits de herramientas de Matplotlib que se encuentran bajo el espacio de nombres ``mpl_toolkits``.
Basemap es un poco torpe de usar, y a menudo incluso las visualizaciones simples toman mucho más tiempo de lo esperado.
Las soluciones más modernas como las API de Google Maps pueden ser una mejor opción para visualizaciones de mapas.
Aún así, Basemap es una herramienta útil para que los usuarios de Python tengan como una de las herramientas virtuales.


La instalación de Basemap es sencilla; si está utilizando conda, puede escribir esto y se descargará el paquete:

`` `
$ conda install basemap
`` `

Agregamos solo una nueva importación a nuestra plantilla estándar:

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap

Los gráficos a continuación también requieren el paquete ``PIL`` en Python 2, o el paquete ``pillow`` en Python 3:

In [ ]:
plt.figure(figsize=(8, 8))
m = Basemap(projection='ortho', resolution=None, lat_0=50, lon_0=-100)
m.bluemarble(scale=0.5);

El significado de los argumentos de `` Basemap``  se discutirá momentáneamente. 

Lo útil es que el globo que se muestra aquí no es una mera imagen; ¡Es un eje Matplotlib totalmente funcional que comprende las coordenadas esféricas y que nos permite superponer fácilmente los datos en el mapa!

Por ejemplo, podemos usar una proyección de mapa diferente, acercarnos a Norteamérica y trazar la ubicación de Seattle.Usaremos una imagen etopo (que muestra características topográficas tanto en tierra como debajo del océano) como fondo del mapa:

In [ ]:
fig = plt.figure(figsize=(8, 8))
m = Basemap(projection='lcc', resolution=None,
            width=8E6, height=8E6, 
            lat_0=45, lon_0=-100,)
m.etopo(scale=0.5, alpha=0.5)

# Map (long, lat) to (x, y) for plotting
x, y = m(-122.3, 47.6)
plt.plot(x, y, 'ok', markersize=5)
plt.text(x, y, ' Seattle', fontsize=12);

Esto le brinda una breve visión del tipo de visualizaciones geográficas que son posibles con solo unas pocas líneas de Python.
Ahora discutiremos las características de Basemap con más profundidad y proporcionaremos varios ejemplos de visualización de datos de mapas.
Usando estos breves ejemplos como bloques de construcción, debería poder crear casi cualquier visualización de mapa que desee.

## Map Projections

Lo primero que debe decidir al usar mapas es qué proyección usar.
Probablemente esté familiarizado con el hecho de que es imposible proyectar un mapa esférico, como el de la Tierra, sobre una superficie plana sin distorsionarlo o romper su continuidad.
Estas proyecciones se han desarrollado a lo largo de la historia humana, ¡y hay muchas opciones!
Dependiendo del uso previsto de la proyección del mapa, hay ciertas características del mapa (por ejemplo, dirección, área, distancia, forma u otras consideraciones) que son útiles para mantener.

El paquete Basemap implementa varias docenas de tales proyecciones, todas referenciadas por un código de formato corto.
Aquí demostraremos brevemente algunos de los más comunes.

Comenzaremos definiendo una rutina de conveniencia para dibujar nuestro mapa mundial junto con las líneas de longitud y latitud:

In [ ]:
from itertools import chain

def draw_map(m, scale=0.2):
    # draw a shaded-relief image
    m.shadedrelief(scale=scale)
    
    # lats and longs are returned as a dictionary
    lats = m.drawparallels(np.linspace(-90, 90, 13))
    lons = m.drawmeridians(np.linspace(-180, 180, 13))

    # keys contain the plt.Line2D instances
    lat_lines = chain(*(tup[1][0] for tup in lats.items()))
    lon_lines = chain(*(tup[1][0] for tup in lons.items()))
    all_lines = chain(lat_lines, lon_lines)
    
    # cycle through these lines and set the desired style
    for line in all_lines:
        line.set(linestyle='-', alpha=0.3, color='w')

### Cylindrical projections

Las proyecciones de mapas más simples son las proyecciones cilíndricas, en las que las líneas de latitud y longitud constantes se asignan a líneas horizontales y verticales, respectivamente. Este tipo de mapeo representa regiones ecuatoriales bastante bien, pero produce distorsiones extremas cerca de los polos.El espaciado de las líneas de latitud varía entre diferentes proyecciones cilíndricas, lo que lleva a diferentes propiedades de conservación y a una distorsión diferente cerca de los polos. 

En la siguiente figura mostramos un ejemplo de la *proyección cilíndrica equidistante*, que elige una escala de latitud que preserva las distancias a lo largo de los meridianos. 

Otras proyecciones cilíndricas son las proyecciones de Mercator (``proyección = 'merc' ``) y las proyecciones cilíndricas de igual área (`` proyección = 'cea' ``).

In [ ]:
fig = plt.figure(figsize=(8, 6), edgecolor='w')
m = Basemap(projection='cyl', resolution=None,
            llcrnrlat=-90, urcrnrlat=90,
            llcrnrlon=-180, urcrnrlon=180, )
draw_map(m)

Los argumentos adicionales para Basemap para esta vista especifican la latitud (`` lat``) y la longitud (`` lon``) de la esquina inferior izquierda (`` llcrnr``) y la esquina superior derecha (`` urcrnr``) para el mapa deseado, en unidades de grados.

### Pseudo-cylindrical projections

Las proyecciones pseudo-cilíndricas relajan el requisito de que los meridianos (líneas de longitud constante) permanezcan verticales; Esto puede dar mejores propiedades cerca de los polos de la proyección.
La proyección de Mollweide (`` proyección = 'moll' ``) es un ejemplo común de esto, en el que todos los meridianos son arcos elípticos.
Está construido para preservar el área del mapa: aunque hay distorsiones cerca de los polos, el área de pequeños parches refleja el área real.
Otras proyecciones pseudo-cilíndricas son las proyecciones sinusoidales (`` proyección = 'sinu' ``) y Robinson (`` proyección = 'robin' ``).

In [ ]:
fig = plt.figure(figsize=(8, 6), edgecolor='w')
m = Basemap(projection='moll', resolution=None,
            lat_0=0, lon_0=0)
draw_map(m)

Los argumentos adicionales para Basemap aquí se refieren a la latitud central (`` lat_0``) y la longitud (`` lon_0``) para el mapa deseado.

### Perspective projections

Las proyecciones de perspectiva se construyen usando una opción particular de punto de perspectiva, similar a si fotografiaras la Tierra desde un punto particular en el espacio (un punto que, para algunas proyecciones, técnicamente se encuentra dentro de la Tierra).
Un ejemplo común es la proyección ortográfica (`` proyección = 'orto' ``), que muestra un lado del globo visto desde un espectador a una distancia muy larga. Como tal, puede mostrar solo la mitad del globo a la vez.
Otras proyecciones basadas en perspectiva incluyen la proyección gnomónica (`` proyección = 'gnom' ``) y la proyección estereográfica (`` proyección = 'stere' ``).
Estos suelen ser los más útiles para mostrar pequeñas porciones del mapa.

Aquí hay un ejemplo de la proyección ortográfica:

In [ ]:
fig = plt.figure(figsize=(8, 8))
m = Basemap(projection='ortho', resolution=None,
            lat_0=50, lon_0=0)
draw_map(m);

### Conic projections

Una proyección cónica proyecta el mapa en un solo cono, que luego se desenrolla.
Esto puede conducir a propiedades locales muy buenas, pero las regiones alejadas del punto de enfoque del cono pueden distorsionarse mucho.
Un ejemplo de esto es la proyección cónica conforme de Lambert (`` proyección = 'lcc' ``), que vimos anteriormente en el mapa de América del Norte.
Proyecta el mapa en un cono dispuesto de tal manera que dos paralelos estándar (especificados en el Mapa base por `` lat_1`` y `` lat_2``) tienen distancias bien representadas, con una escala que disminuye entre ellos y aumenta fuera de ellos.
Otras proyecciones cónicas útiles son la proyección cónica equidistante (`` proyección = 'eqdc' ``) y la proyección de área igual de Albers (`` proyección = 'aea' ``).
Las proyecciones cónicas, como las proyecciones en perspectiva, tienden a ser buenas opciones para representar parches pequeños o medianos del globo.

In [ ]:
fig = plt.figure(figsize=(8, 8))
m = Basemap(projection='lcc', resolution=None,
            lon_0=0, lat_0=50, lat_1=45, lat_2=55,
            width=1.6E7, height=1.2E7)
draw_map(m)

### Other projections

Si va a utilizar mucho visualizaciones basadas en mapas, le recomiendo que lea sobre otras proyecciones disponibles, junto con sus propiedades, ventajas y desventajas.
Lo más probable es que estén disponibles en el [Paquete de mapa base](http://matplotlib.org/basemap/users/mapsetup.html).

## Dibujar un mapa como fondo

Anteriormente vimos los métodos `` bluemarble () `` y `` shadedrelief () `` para proyectar imágenes globales en el mapa, así como los métodos `` drawparallels () `` y `` drawmeridians () `` para dibujar líneas de latitud y longitud constantes.
El paquete de Basemap contiene una gama de funciones útiles para dibujar fronteras de características físicas como continentes, océanos, lagos y ríos, así como límites políticos como países y estados y condados de EE. UU.
Las siguientes son algunas de las funciones de dibujo disponibles que puede explorar utilizando las funciones de ayuda de IPython:

- ** Límites físicos y cuerpos de agua **
  - `` drawcoastlines () ``: dibuja líneas de costa continental
  - `` drawlsmask () ``: dibuja una máscara entre la tierra y el mar, para usar con la proyección de imágenes en uno u otro
  - `` drawmapboundary () ``: dibuja el límite del mapa, incluido el color de relleno para los océanos.
  - `` drawrivers () ``: dibuja ríos en el mapa
  - `` fillcontinents () ``: Rellena los continentes con un color dado; opcionalmente llenar lagos con otro color

- ** Límites políticos **
  - `` drawcountries () ``: dibuja los límites del país
  - `` drawstates () ``: dibuja los límites del estado de EE. UU.
  - `` drawcounties () ``: dibuja los límites de los condados de EE. UU.

- ** Características del mapa **
  - `` drawgreatcircle () ``: dibuja un gran círculo entre dos puntos
  - `` drawparallels () ``: Dibuja líneas de latitud constante
  - `` drawmeridians () ``: dibuja líneas de longitud constante
  - `` drawmapscale () ``: dibuja una escala lineal en el mapa

- ** Imágenes de globo completo **
  - `` bluemarble () ``: proyecte la imagen de mármol azul de la NASA en el mapa
  - `` sombreado () '': proyecta una imagen de relieve sombreada en el mapa
  - `` etopo () ``: dibuja una imagen de relieve de etopo en el mapa
  - `` warpimage () ``: proyecta una imagen proporcionada por el usuario en el mapa

Para las características basadas en límites, debe establecer la resolución deseada al crear una imagen de mapa base.
El argumento `` resolución `` de la clase `` Basemap `` establece el nivel de detalle en los límites, ya sea `` 'c'`` (crudo), ``' l'`` (bajo), `` 'i `` '(intermedio), ``' h'`` (alto), `` 'f' `` (completo) o `` Ninguno `` si no se utilizarán límites.
Esta elección es importante: establecer límites de alta resolución en un mapa global, por ejemplo, puede ser *muy* lento.

Aquí hay un ejemplo de cómo dibujar límites terrestres / marítimos y el efecto del parámetro de resolución.
Crearemos un mapa de baja y alta resolución de la hermosa isla de Skye en Escocia.
Está ubicado a 57.3 ° N, 6.2 ° O, y un mapa de 90,000 × 120,000 kilómetros lo muestra bien:

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 8))

for i, res in enumerate(['l', 'h']):
    m = Basemap(projection='gnom', lat_0=57.3, lon_0=-6.2,
                width=90000, height=120000, resolution=res, ax=ax[i])
    m.fillcontinents(color="#FFDDCC", lake_color='#DDEEFF')
    m.drawmapboundary(fill_color="#DDEEFF")
    m.drawcoastlines()
    ax[i].set_title("resolution='{0}'".format(res));

Tenga en cuenta que las costas de baja resolución no son adecuadas para este nivel de zoom, mientras que la alta resolución funciona bien.
Sin embargo, el nivel bajo funcionaría bien para una vista global y sería *mucho* más rápido que cargar los datos de borde de alta resolución para todo el mundo.
Puede que sea necesario experimentar un poco para encontrar el parámetro de resolución correcto para una vista determinada: la mejor ruta es comenzar con un trazado rápido y de baja resolución y aumentar la resolución según sea necesario.

## Plotting Data on Maps

Quizás la pieza más útil del kit de herramientas de Basemap es la capacidad de sobregraficar una variedad de datos en un fondo de mapa.
Para trazado y texto simples, cualquier función `` plt`` funciona en el mapa; puede usar la instancia de `` Basemap `` para proyectar las coordenadas de latitud y longitud a las coordenadas `` (x, y) `` para trazar con `` plt``, como vimos anteriormente en el ejemplo de Seattle.

Además de esto, hay muchas funciones específicas de mapas disponibles como métodos de la instancia de `` Basemap ``.
Estos funcionan de manera muy similar a sus contrapartes estándar de Matplotlib, pero tienen un argumento booleano adicional `` latlon``, que si se establece en `` True `` le permite pasar latitudes y longitudes sin procesar al método, en lugar de proyectarse `` (x , y) `` coordenadas.

Algunos de estos métodos específicos del mapa son:

- `` contour () `` / `` contourf () ``: dibuja líneas de contorno o contornos rellenos
- `` imshow () ``: dibuja una imagen
- `` pcolor () `` / `` pcolormesh () ``: Dibuja un diagrama de pseudocolor para mallas irregulares / regulares
- `` plot () ``: Dibuja líneas y / o marcadores.
- `` scatter () ``: Dibuja puntos con marcadores.
- `` quiver () ``: Dibuja vectores.
- `` barbs () ``: dibuja púas de viento.
- `` drawgreatcircle () ``: Dibuja un gran círculo.

Veremos algunos ejemplos de algunos de estos a medida que continuamos.
Para obtener más información sobre estas funciones, incluidas varias parcelas de ejemplo, consulte la [documentación del mapa base en línea](http://matplotlib.org/basemap/).

## Example: California Cities

In [ ]:
import pandas as pd
cities = pd.read_csv('data/california_cities.csv')

# Extract the data we're interested in
lat = cities['latd'].values
lon = cities['longd'].values
population = cities['population_total'].values
area = cities['area_total_km2'].values

A continuación, configuramos la proyección del mapa, dispersamos los datos y luego creamos una barra de colores y una leyenda:

In [ ]:
# 1. Draw the map background
fig = plt.figure(figsize=(8, 8))
m = Basemap(projection='lcc', resolution='h', 
            lat_0=37.5, lon_0=-119,
            width=1E6, height=1.2E6)
m.shadedrelief()
m.drawcoastlines(color='gray')
m.drawcountries(color='gray')
m.drawstates(color='gray')

# 2. scatter city data, with color reflecting population
# and size reflecting area
m.scatter(lon, lat, latlon=True,
          c=np.log10(population), s=area,
          cmap='Reds', alpha=0.5)

# 3. create colorbar and legend
plt.colorbar(label=r'$\log_{10}({\rm population})$')
plt.clim(3, 7)

# make legend with dummy points
for a in [100, 300, 500]:
    plt.scatter([], [], c='k', alpha=0.5, s=a,
                label=str(a) + ' km$^2$')
plt.legend(scatterpoints=1, frameon=False,
           labelspacing=1, loc='lower left');

Esto nos muestra aproximadamente dónde se han establecido poblaciones más grandes de personas en California: se agrupan cerca de la costa en las áreas de Los Ángeles y San Francisco, se extienden a lo largo de las carreteras en el valle central plano y evitan casi por completo las regiones montañosas a lo largo de las fronteras de el estado.

## Example: Surface Temperature Data

Como ejemplo de visualización de algunos datos geográficos más continuos, consideremos el "vórtice polar" que golpeó la mitad oriental de los Estados Unidos en enero de 2014.
Una gran fuente para cualquier tipo de datos climáticos es [el Instituto Goddard de Estudios Espaciales de la NASA](http://data.giss.nasa.gov/).
Aquí usaremos los datos de temperatura GIS 250, que podemos descargar usando comandos de shell (estos comandos pueden tener que modificarse en máquinas con Windows).
Los datos utilizados aquí se descargaron el 12/06/2016 y el tamaño del archivo es de aproximadamente 9 MB:

In [ ]:
# !curl -O http://data.giss.nasa.gov/pub/gistemp/gistemp250.nc.gz
# !gunzip gistemp250.nc.gz

Los datos vienen en formato NetCDF, que puede leerse en Python mediante la biblioteca `` netCDF4``.
Puede instalar esta biblioteca como se muestra aquí

`` `
$ conda instalar netcdf4
`` `

Leemos los datos de la siguiente manera:

In [ ]:
from netCDF4 import Dataset
data = Dataset('gistemp250.nc')

El archivo contiene muchas lecturas de temperatura global en una variedad de fechas; debemos seleccionar el índice de la fecha que nos interesa, en este caso, el 15 de enero de 2014:

In [ ]:
from netCDF4 import date2index
from datetime import datetime
timeindex = date2index(datetime(2014, 1, 15),
                       data.variables['time'])

Ahora podemos cargar los datos de latitud y longitud, así como la anomalía de temperatura para este índice:

In [ ]:
lat = data.variables['lat'][:]
lon = data.variables['lon'][:]
lon, lat = np.meshgrid(lon, lat)
temp_anomaly = data.variables['tempanomaly'][timeindex]

Finalmente, usaremos el método `` pcolormesh () `` para dibujar una malla de color de los datos.
Miraremos a América del Norte y utilizaremos un mapa de relieve sombreado en el fondo.
Tenga en cuenta que para estos datos elegimos específicamente un mapa de color divergente, que tiene un color neutro en cero y dos colores contrastantes en valores negativos y positivos.
También dibujaremos ligeramente las costas sobre los colores como referencia:

In [ ]:
fig = plt.figure(figsize=(10, 8))
m = Basemap(projection='lcc', resolution='c',
            width=8E6, height=8E6, 
            lat_0=45, lon_0=-100,)
m.shadedrelief(scale=0.5)
m.pcolormesh(lon, lat, temp_anomaly,
             latlon=True, cmap='RdBu_r')
plt.clim(-8, 8)
m.drawcoastlines(color='lightgray')

plt.title('January 2014 Temperature Anomaly')
plt.colorbar(label='temperature anomaly (°C)');

Los datos pintan una imagen de las anomalías localizadas de temperatura extrema que ocurrieron durante ese mes.
La mitad oriental de los Estados Unidos era mucho más fría de lo normal, mientras que la mitad occidental y Alaska eran mucho más cálidos.
Las regiones sin temperatura registrada muestran el fondo del mapa.